# Python based analysis

This script takes trees as inputs and writes another sets of trees as outputs.<br>
The trees are filtered in between.

In [1]:
import os, sys
import numpy as np
import uproot
import pandas as pd

In [2]:
#Global parameters

basedir = '../input_trees_modified'
injob  = 'tree_2LSSinclusive_baseline_Sept17_test'
outjob = 'tree_2LSS_qcdVR_Sept25'
channelno = 0 #mumu

indir = os.path.join(basedir, injob)
outdir = os.path.join(basedir, outjob)
os.makedirs(outdir, exist_ok=True) 

In [3]:
def read_file_into_df(filepath):

    tfile = uproot.open(filepath)
    ttree = tfile['myEvents']
    branches = ttree.keys()
    awkarray = ttree.arrays(branches)
    df = pd.DataFrame(awkarray.to_list())

    if 'channel' in df.columns:
        df = df[df['channel'] == channelno]
        
    #print(f'dataframe for {sample} {subsample} loaded.')
    return df

def write_df_into_file(df, filepath):
    if df.empty:
        data_dict = {col: np.array([], dtype=df[col].dtype) for col in df.columns}
        print(f"\033[0;31mWarning: Writing empty file: {filepath}\033[0m\n")
    else:
        data_dict = df.to_dict('list')
        
    with uproot.recreate(filepath) as file: file['myEvents'] = data_dict
    
print('Functions loaded.')

Functions loaded.


In [4]:
%%time

list_of_files = os.listdir(indir)
list_empty_input = []
list_empty_output = []

for f in list_of_files:

    #Step1: Prepare the dataframe
    print(f'\nLoading file: {f}')
    filepath = os.path.join(indir, f)
    sample = f.split("_")[1]
    subsample = "_".join(f.split("_")[2:])
       
    outfile = os.path.join(outdir, f)
    #if os.path.exists(outfile): continue
    #if 'WJets' not in f: continue

    df = read_file_into_df(filepath)
    if df.empty : 
        print(f"\033[0;31mWarning: Empty input file: {filepath}\033[0m\n")
        list_empty_input.append(filepath)

    # Put any cuts here on dataframe as you like
    # Define conditions as strings
    nncut = 'nnscore_qcd_vlldmu < 0.25'
    qcdcr = '0.02<lep0_iso<0.15 and lep0_sip3d>5'
    qcdvr = '0.02<lep1_iso<0.15 and lep1_sip3d>5'

    # Combine conditions for query
    #event_selection = f"{nncut} and {qcdcr}"
    event_selection = f"{nncut} and {qcdvr} and not {qcdcr}"

    # Use df.query with the combined condition
    df_filtered = df.query(event_selection)
    
    if df_filtered.empty:
        list_empty_output.append(f'{sample}_{subsample}')

    nbefore = len(df)
    nafter = len(df_filtered)

    frac = 0
    if nbefore != 0: frac = nafter*100/nbefore
    
    write_df_into_file(df_filtered, outfile)
    print(f'\033[1;32mFile written: {outfile}\033[0m ({nbefore}->{nafter}, {frac:.2f}%)\n')

print(f'\n\033[1;33mSummary:\033[0m')
print(f'\n\033[33mEmpty input: {list_empty_input}\033[0m')
print(f'\n\033[31mEmpty output: {list_empty_output}\033[0m\n')


Loading file: tree_DYJetsToLL_M10to50.root

File written: ../input_trees_modified/tree_2LSS_qcdVR_Sept25/tree_DYJetsToLL_M10to50.root (44->0, 0.00%)


Loading file: tree_DYJetsToLL_M50.root
File written: ../input_trees_modified/tree_2LSS_qcdVR_Sept25/tree_DYJetsToLL_M50.root (1974->5, 0.25%)


Loading file: tree_EGamma_EGamma_A.root

File written: ../input_trees_modified/tree_2LSS_qcdVR_Sept25/tree_EGamma_EGamma_A.root (8->0, 0.00%)


Loading file: tree_EGamma_EGamma_B.root

File written: ../input_trees_modified/tree_2LSS_qcdVR_Sept25/tree_EGamma_EGamma_B.root (7->0, 0.00%)


Loading file: tree_EGamma_EGamma_C.root

File written: ../input_trees_modified/tree_2LSS_qcdVR_Sept25/tree_EGamma_EGamma_C.root (1->0, 0.00%)


Loading file: tree_EGamma_EGamma_D.root

File written: ../input_trees_modified/tree_2LSS_qcdVR_Sept25/tree_EGamma_EGamma_D.root (23->0, 0.00%)


Loading file: tree_Higgs_bbH_HToZZTo4L.root

File written: ../input_trees_modified/tree_2LSS_qcdVR_Sept25/tree_Higgs_bbH_HToZZT

### Convert them into histograms

In [5]:
%%time

run_here = False

channel_ = 'mm'
jobname_ = outjob
dump_ = outjob.replace('tree', 'hist')+'_'+channel_
command = f'python3 extractHistFromTree.py --jobname {jobname_} --dump {dump_} --channel {channel_}'
print(command)

if run_here: os.system(command)
else: print('Run it yourself:')
    
print('Done!')

python3 extractHistFromTree.py --jobname tree_2LSS_qcdVR_Sept25 --dump hist_2LSS_qcdVR_Sept25_mm --channel mm
Run it yourself:
Done!
CPU times: user 146 μs, sys: 0 ns, total: 146 μs
Wall time: 138 μs
